In [2]:
%matplotlib inline

import os 
import sys 
import time
import hashlib

from datetime  import datetime 

import numpy as np  # learn 
import pandas as pd # learn
from pandas import *
from numpy import *

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *

 
import matplotlib as mplt # learn matplolib 
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

import seaborn as sns 
sns.set_style("whitegrid")
sns.set(rc={"figure.figsize": (14, 6)})

import IPython
from IPython.display import display


import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
 
import sklearn as sk
import itertools



from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from pyspark.mllib.regression import LabeledPoint


In [2]:
# data preparison: load data
fdta_RDD = sc.textFile("file:///home/tguo/data/tian-test/2008_mllib.csv")

# dta_cateFeature_resetIdxAndFilter= sc.textFile(
#     "hdfs://computer61.ant-net/user/tguo/tian_udpjitter-H1april2014_regTree_mllib.csv")

print 'loaded data'
print type(fdta_RDD)

fdta_RDD.take(1)

loaded data
<class 'pyspark.rdd.RDD'>


[u'-14.0,1,3,4,0,0,0,0']

In [7]:
# fdta_ready = fdta_RDD.map(lambda line: line.split(",")).map(lambda r:(float(r[0]),float(r[1]),float(r[2]),float(r[3]),float(r[4]),float(r[5]),float(r[6]),float(r[7])))

fdta_ready = fdta_RDD.map(lambda line: line.split(",")).map(lambda r:(float(r[0]),float(r[1]),float(r[2]),float(r[3]),float(r[4]),float(r[5]),
                                                                      float(r[6]),float(r[7])))


fdta_ready.take(2)

[(-14.0, 1.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0),
 (2.0, 1.0, 3.0, 4.0, 1.0, 0.0, 0.0, 0.0)]

In [8]:
#model training: cross-validation for regression tree with categorical features
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
flabeledData=  fdta_ready.map(lambda line: LabeledPoint(line[1],[line[1],line[2],line[3],line[4],line[5],line[6],line[7]]))
depth_up=3
depth_low=0

# flabeledData=flabeledData.filter(lambda line: line.label>0 )

test_error=[]
training_error=[]

(ftrainingData,ftestData)=flabeledData.randomSplit([0.8, 0.2])

ftest_labels = ftestData.map(lambda lp: lp.label)
local_test_labels = ftest_labels.collect()

ftrain_labels = ftrainingData.map(lambda lp: lp.label)
local_train_labels = ftrain_labels.collect()

test_size= len(local_test_labels) 
train_size= len(local_train_labels) 


for depth in range(depth_low, depth_up):
    model=DecisionTree.trainRegressor(ftrainingData,impurity='variance',maxDepth=depth,maxBins=7600,
                                      categoricalFeaturesInfo={0:13,1:32,2:8,3:7539,4:20,5:303,6:304 })
    
    test_predictions = model.predict(ftestData.map(lambda x: x.features))
    local_test_predictions = test_predictions.collect()
    
    
    train_predictions = model.predict(ftrainingData.map(lambda x: x.features))
    local_train_predictions = train_predictions.collect()


    # test error
    local_test_resid=map( lambda x,y: (x-y)*(x-y), local_test_predictions,local_test_labels )
    test_error.append( sum(local_test_resid)/test_size  )
    
    #training error 
    local_train_resid=map( lambda x,y: (x-y)*(x-y), local_train_predictions,local_train_labels )
    training_error.append( sum(local_train_resid)/train_size ) 
    
    print depth,"depth:", sum(local_test_resid)/test_size,sum(local_train_resid)/train_size
    print depth,model

0 depth: 11.6188529515 11.6026038776
0 DecisionTreeModel regressor of depth 0 with 1 nodes
1 depth: 2.95040936769 2.94747018771
1 DecisionTreeModel regressor of depth 1 with 3 nodes
2 depth: 0.670535696322 0.669452528488
2 DecisionTreeModel regressor of depth 2 with 7 nodes


In [18]:
ftrainingData.take(2)

[LabeledPoint(2.0, [1.0,3.0,4.0,1.0,0.0,0.0,0.0]),
 LabeledPoint(14.0, [1.0,3.0,4.0,2.0,0.0,1.0,1.0])]

In [6]:
# test for mllib svm dataset
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, 'file:///home/tguo/sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.8, 0.2])

# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.


for depth in range(0,2):
    model = DecisionTree.trainRegressor(trainingData, categoricalFeaturesInfo={},
                                    impurity='variance', maxDepth=depth, maxBins=64)

    # Evaluate model on test instances and compute test error
    predictions = model.predict(testData.map(lambda x: x.features))
    labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
    testMSE = labelsAndPredictions.map(lambda (v, p): (v - p) * (v - p)).sum() / float(testData.count())
    print('Test Mean Squared Error = ' + str(testMSE))
# print('Learned regression tree model:')
# print(model.toDebugString())

# Save and load model
# model.save(sc, "myModelPath")
# sameModel = DecisionTreeModel.load(sc, "myModelPath")

Test Mean Squared Error = 0.24245412707
Test Mean Squared Error = 0.0454545454545


In [16]:
# tmprdd=trainingData.filter(lambda line: line.label>0)
# tmprdd.count()
# tmprdd.take(44)

In [3]:
tmpdta=pd.read_csv('/home/tguo/tian_src/syndata_set/3.csv')

In [11]:
tmpdta['DISTRICT_NAME'][1:10]

1    NORTHEAST VERNON CO. R-I
2              KIRKWOOD R-VII
3           CLINTON CO. R-III
4            PATTONSBURG R-II
5            GRAIN VALLEY R-V
6              CALHOUN R-VIII
7              MEHLVILLE R-IX
8               BOONVILLE R-I
9                 SEDALIA 200
Name: DISTRICT_NAME, dtype: object

In [14]:
tmpdta[ tmpdta['DISTRICT_NAME'] == 'NORTHEAST VERNON CO. R-I' ][  ['SaidStudent', 'YEAR']    ]

,SaidStudent,YEAR
0,2706.827,2007
1,3060.870,2008
79,3529.126,2009
748,2689.189,2010
1295,2578.512,2011
1404,2565.056,2012
